---
### 크롤링
https://astro.kasi.re.kr/life/pageView/9?lat=35.735786158101774

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import re
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote_plus

In [ ]:
date = ['2020-01-01', '2020-01-10', '2020-01-20', '2020-01-30', '2020-02-01', '2020-02-10', '2020-02-20', '2020-02-29', 
        '2020-03-01', '2020-03-10', '2020-03-20', '2020-03-30', '2020-04-01', '2020-04-10', '2020-04-20', '2020-04-30',
        '2020-05-01', '2020-05-10', '2020-05-20', '2020-05-30', '2020-06-01', '2020-06-10', '2020-06-20', '2020-06-30',
        '2020-07-01', '2020-07-10', '2020-07-20', '2020-07-30', '2020-08-01', '2020-08-10', '2020-08-20', '2020-08-30',
        '2020-09-01', '2020-09-10', '2020-09-20', '2020-09-30', '2020-10-01', '2020-10-10', '2020-10-20', '2020-10-30',
        '2020-11-01', '2020-11-10', '2020-11-20', '2020-11-30', '2020-12-01', '2020-12-10', '2020-12-20', '2020-12-30']

sunrise = []; sunset = []; daytime = []


driver = webdriver.Chrome("./chromedriver")
for i in range(len(date)):
    add = 'https://astro.kasi.re.kr/life/pageView/9?lat=37.59128838881846&lng=127.02096562433636&date='+ date[i] + '&address=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%84%B1%EB%B6%81%EA%B5%AC+%EB%B3%B4%EB%AC%B8%EB%A1%9C34%EB%8B%A4%EA%B8%B8+2'
    driver.get(add)
    time.sleep(1)
    sr = driver.find_element_by_css_selector("#cont_skip > div.search_box > div > div:nth-child(1) > p > span").text    ; sunrise.append(sr)
    ss = driver.find_element_by_css_selector('#cont_skip > div.search_box > div > div:nth-child(3) > p > span').text    ; sunset.append(ss)
    dt = driver.find_element_by_css_selector("#daylength").text; daytime.append(dt)

driver.close() 

In [ ]:
data = pd.DataFrame({'date':date, 'sunrise': sunrise, 'sunset': sunset, 'daytime': daytime})
data.tail() 

,date,sunrise,sunset,daytime
43,2020-11-30,07시 27분,17시 14분,09시간 46분
44,2020-12-01,07시 28분,17시 13분,09시간 45분
45,2020-12-10,07시 35분,17시 13분,09시간 37분
46,2020-12-20,07시 42분,17시 16분,09시간 33분
47,2020-12-30,07시 46분,17시 22분,09시간 35분


---
### 네이버웹툰 댓글
댓글 사이트가 따로 있음! 

In [ ]:
# 똑똑한 나 인증 - 광고웹툰
URL = 'https://comic.naver.com/comment/comment.nhn?titleId=766658&no=1'
html=requests.get(URL).text
soup=BeautifulSoup(html,'html.parser')
time.sleep(0.4)

driver = webdriver.Chrome('chromedriver.exe')
driver.get(URL)

# 댓글 설정 > 전체 댓글 & 클린봇 해제
driver.find_element_by_partial_link_text('전체 댓글 더보기').click()
driver.find_element_by_css_selector('#cbox_module > div > div.u_cbox_cleanbot > a').click(); time.sleep(0.3)
driver.find_element_by_css_selector('#cleanbot_dialog_checkbox_cbox_module').click(); time.sleep(0.3)
driver.find_element_by_css_selector('body > div.u_cbox.u_cbox_layer_wrap > div > div.u_cbox_layer_cleanbot > div.u_cbox_layer_cleanbot_extra > a').click() 


#### 긁어오는거 함수로 ###########################################################
def webtoon_crawling():
    idd = driver.find_elements_by_class_name('u_cbox_info');                 time.sleep(0.5) # 아이디
    date = driver.find_elements_by_class_name('u_cbox_date');                time.sleep(0.3) # 댓글 작성 날짜
    chat = driver.find_elements_by_class_name('u_cbox_contents');            time.sleep(0.5) # 댓글
    recom = driver.find_elements_by_class_name('u_cbox_cnt_recomm');         time.sleep(0.3) # 추천 수
    unrecom = driver.find_elements_by_class_name('u_cbox_cnt_unrecomm');     time.sleep(0.5) # 비추천 수

    while len(idd)>0:
        id_list.append(idd[0].text) # 아이디
        date_list.append(date[0].text) # 날짜
        chat_list.append(chat[0].text) # 댓글
        recom_list.append(recom[0].text) # 추천 수
        unrecom_list.append(unrecom[0].text) # 비추천 수

        idd.pop(0) ; date.pop(0); chat.pop(0); recom.pop(0); unrecom.pop(0)

    time.sleep(0.1)
#################################################################################

# 아이디, 날짜, 댓글, 추천 수, 비추천 수
id_list=[] ; date_list=[] ; chat_list=[]; recom_list = []; unrecom_list = [] 
num=0 ; start=0 ; error_back=0

while True:
    try:
        while start<=0:
            time.sleep(0.4)
            # 댓글 페이지
            page = driver.find_elements_by_class_name('u_cbox_num_page')
            print('now page: ',int(page[num].text),' index: ',num);                  time.sleep(0.5)
    
            webtoon_crawling()  ## 함수 사용
            time.sleep(0.3)
            
            #페이지가 10의 배수이면 다음 페이지 클릭, 동일 과정 거침
            try:
                a = driver.find_elements_by_class_name('u_cbox_next') #*****CLEAR v
                if(int(page[num].text) % 10 == 0 and a[0].text != ''):
                    driver.find_element_by_class_name('u_cbox_next').click()
                    time.sleep(0.5)
                    # 다음 페이지
                    page = driver.find_elements_by_class_name('u_cbox_num_page')
                    time.sleep(0.3)                
                    webtoon_crawling()  ## 함수 사용
    
                    num=0
                    print('now page: ',int(page[num].text),' index: ',num)
            
            except IndexError: break
                
            num +=1
            time.sleep(0.4)
            
            try:
                page[num].click()
            except IndexError:
                break
            
            page.clear()
            idd.clear()
            time.sleep(1)
        
        # 페이지 닫기
        error_back=0
        driver.quit()
        break
        
    except:
        print('ERROR')
        error_back +=1
        if(error_back>10):
            page[num].click()
        continue

now page:  1  index:  0
now page:  2  index:  1
now page:  3  index:  2
now page:  4  index:  3
now page:  5  index:  4
now page:  6  index:  5
now page:  7  index:  6
now page:  8  index:  7
now page:  9  index:  8
now page:  10  index:  9
ERROR
now page:  20  index:  9
now page:  21  index:  0
now page:  22  index:  1
now page:  23  index:  2
now page:  24  index:  3
now page:  25  index:  4
now page:  26  index:  5
now page:  27  index:  6
now page:  28  index:  7
now page:  29  index:  8
now page:  30  index:  9
ERROR
now page:  30  index:  9
now page:  31  index:  0
now page:  32  index:  1
now page:  33  index:  2
now page:  34  index:  3
now page:  35  index:  4
now page:  36  index:  5
now page:  37  index:  6
now page:  38  index:  7
now page:  39  index:  8
now page:  40  index:  9
ERROR
now page:  41  index:  0
now page:  42  index:  1
now page:  43  index:  2
now page:  44  index:  3
now page:  45  index:  4
now page:  46  index:  5
now page:  47  index:  6
now page:  48  i

In [ ]:
df = pd.DataFrame({'아이디': id_list, '작성일': date_list, '댓글': chat_list, '추천 수': recom_list, '비추천 수': unrecom_list})
df.to_csv('똑똑한 나 인증.csv') 
pd.read_csv('똑똑한 나 인증.csv') 

,Unnamed: 0,아이디,작성일,댓글,추천 수,비추천 수
0,0,아\n(spdl****),13시간 전,모죠,0,0
1,1,레이나\n(mir4****),1일 전,공인 인증서 진짜 벼어어어엉시이이이인임,0,7
2,2,레이나\n(mir4****),1일 전,공인 인증서 진짜 벼어어어엉시이이이인임,0,7
3,3,yuc\n(yuch****),1일 전,ㅗㅜㅗㅜ 싸이버펑크 2021인가?,0,0
4,4,조수빈\n(subi****),2일 전,나만 스크롤 내리다가 들어온게 아니구나 ㅋㅋㅋㅋㅋㅋㅋ 묘조님 그림이 넘 반갑고 눈에...,1,0
...,...,...,...,...,...,...
1468,1468,꼽냐\n(siny****),2021-02-11 22:02,어 이게 뭐얔ㅋㅋㅋㅋ,2,0
1469,1469,유토끼\n(keyt****),2021-02-11 22:02,난 약 1초전에 웹툰이 생기는걸 보았다....,10,3
1470,1470,고독의 미\n(zaca****),2021-02-11 22:01,모죠..?,2,0
1471,1471,js1\n(js11****),2021-02-11 22:01,오늘밤 주인공은 나야나,1,0
